## Confidentiality

This notebook is downloaded from Tensorflow and is for demonstrational purposes only.

Please do not copy or distribute this notebook.

# Loading text

Here's described how to use `tf.data.TextLineDataset` to load examples from text files. `TextLineDataset` is designed to create a dataset from a text file, in which each example is a line of text from the original file. This is potentially useful for any text data that is primarily line-based (for example, management reports or error logs).

Here, three different English translations are used of the same text. A model is trained to identify the translator given a single line of text.

## Setup

In [ ]:
!pip install -q tfds-nightly

     |████████████████████████████████| 3.6MB 3.3MB/s 


In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
import json, math, os, sys

The texts of the three translations are by:

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

The text files have undergone some typical preprocessing tasks, mostly removing stuff — document header and footer, line numbers, chapter titles. Downloading these preprocessed files locally.

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

## Loading text into datasets

Iterating through the files, loading each one into its own dataset.

Each example needs to be individually labeled, so using `tf.data.Dataset.map` to apply a labeler function to each one. This will iterate over every example in the dataset, returning (`example, label`) pairs.

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

Combining these labeled datasets into a single dataset, and shuffling it.


In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

You can use `tf.data.Dataset.take` and `print` to see what the `(example, label)` pairs look like. The `numpy` property shows each Tensor's value.

In [ ]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Of Ilium and his host shall perish all.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'sat apart in his all-glorious majesty, looking down upon the city of'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Down from his chariot with his arms he leap'd,">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Whereon his incense-honour'd altar stood:">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Of Peleus' godlike son, that thy own ship">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


## Encoding text lines as numbers

Machine learning models work on numbers, not words, so the string values need to be converted into lists of numbers. To do that, map each unique word to a unique integer.

### Building vocabulary

First, building a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python:

1. Iterate over each example's `numpy` value.
2. Use `tfds.deprecated.text.Tokenizer` to split it into tokens.
3. Collect these tokens into a Python set, to remove duplicates.
4. Get the size of the vocabulary for later use.

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

### Encoding examples

Creating an encoder by passing the `vocabulary_set` to `tfds.deprecated.text.TokenTextEncoder`. The encoder's `encode` method takes in a string of text and returns a list of integers.

In [ ]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

Trying this on a single line to see what the output looks like.

In [ ]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'Of Ilium and his host shall perish all.'


In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[8488, 11525, 14014, 711, 1682, 11795, 13485, 9832]


Running the encoder on the dataset by wrapping it in `tf.py_function` and  passing that to the dataset's `map` method.

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

Using `Dataset.map` to apply this function to each element of the dataset.  `Dataset.map` runs in graph mode.

* Graph tensors do not have a value. 
* In graph mode you can only use TensorFlow Ops and functions. 

Wrapping it in a `tf.py_function`. The `tf.py_function` will pass regular tensors (with a value and a `.numpy()` method to access it), to the wrapped python function.

In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so setting the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## Splitting the dataset into test and train batches

Using `tf.data.Dataset.take` and `tf.data.Dataset.skip` to create a small test dataset and a larger training set.

Before being passed into the model, the datasets need to be batched. Typically, the examples inside of a batch need to be the same size and shape. But, the examples in these datasets are not all the same size — each line of text had a different number of words. So using `tf.data.Dataset.padded_batch` (instead of `batch`) to pad the examples to the same size.

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Now, `test_data` and `train_data` are not collections of (`example, label`) pairs, but collections of batches. Each batch is a pair of (*many examples*, *many labels*) represented as arrays.

To illustrate:

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=
 array([ 8488, 11525, 14014,   711,  1682, 11795, 13485,  9832,     0,
            0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

Since there's introduced a new token encoding (the zero used for padding), the vocabulary size has increased by one.

In [ ]:
vocab_size += 1

## Building the model


In [ ]:
model = tf.keras.Sequential()

The first layer converts integer representations to dense vector embeddings. See the [word embeddings tutorial](../text/word_embeddings.ipynb) or more details. 

In [ ]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

The next layer is a [Long Short-Term Memory](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) layer, which lets the model understand words in their context with other words. A bidirectional wrapper on the LSTM helps it to learn about the datapoints in relationship to the datapoints that came before it and after it.

In [ ]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Finally there's a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.

In [ ]:
# One or more dense layers.
# Editing the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

Finally, compiling the model. For a softmax categorization model, using `sparse_categorical_crossentropy` as the loss function. You can try other optimizers, but `adam` is very common.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Training the model

This model running on this data produces decent results (about 83%).

In [ ]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 27s 38ms/step - loss: 0.4976 - accuracy: 0.7629 - val_loss: 0.3920 - val_accuracy: 0.8200
Epoch 2/3
697/697 [==============================] - 26s 37ms/step - loss: 0.2916 - accuracy: 0.8739 - val_loss: 0.3695 - val_accuracy: 0.8320
Epoch 3/3
697/697 [==============================] - 26s 37ms/step - loss: 0.2187 - accuracy: 0.9050 - val_loss: 0.3940 - val_accuracy: 0.8322


In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 2s 25ms/step - loss: 0.3940 - accuracy: 0.8322

Eval loss: 0.394, Eval accuracy: 0.832
